In [24]:
import sys
sys.path.append('/Users/syesw/Desktop/My_Code_Project/Fintech-Paper/[1] neural_network/cnn-candle-pattern-predict')

In [25]:
from sklearn.metrics import confusion_matrix
import numpy as np

from keras import backend as K
from keras import optimizers
from tensorflow.python.keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, Activation

from utils import util_processor as pro


In [26]:
class CNNModelTrainer:
    def __init__(self, params):
        self.params = params # 超参数
        self.model = None # CNN模型

    def get_model(self):
        model = Sequential()

        # 构建模型
        # 16个卷积核，大小为2x2，步长为1x1，padding为same(这会导致卷积层输出相同的尺寸)
        model.add(Conv2D(16, (2, 2), input_shape=(10, 10, 4), padding='same', strides=(1, 1)))
        model.add(Activation('sigmoid'))
        model.add(Conv2D(16, (2, 2), padding='same', strides=(1, 1)))
        model.add(Activation('sigmoid'))
        model.add(Flatten()) # flatten后展开为8*8*16=1024向量

        # 全连接层
        model.add(Dense(128, activation='relu'))
        model.add(Dense(self.params['classes']))
        model.add(Activation('softmax')) # 转化为概率
        model.summary()

        self.model = model
        return self.model

    def train_model(self, data):
        model = self.get_model()
        model.compile(loss='categorical_crossentropy', optimizer=self.params['optimizer'], metrics=['accuracy'])
        hist = model.fit(x=data['train_gaf'], y=data['train_label_arr'],
                         validation_data=(data['val_gaf'], data['val_label_arr']),
                         batch_size=self.params['batch_size'], epochs=self.params['epochs'], verbose=2)
        
        return (model, hist)

    def print_result(self, data):
        # get train & test pred-labels

        train_pred = np.argmax(self.model.predict(data['train_gaf']),axis=1)
        test_pred = np.argmax(self.model.predict(data['test_gaf']),axis=1)
        #train_pred = self.model.predict_classes(data['train_gaf'])
        #test_pred = self.model.predict_classes(data['test_gaf'])
        
        # get train & test true-labels
        train_label = data['train_label'][:, 0]
        test_label = data['test_label'][:, 0]
        # train & test confusion matrix
        train_result_cm = confusion_matrix(train_label, train_pred, labels=range(9))
        test_result_cm = confusion_matrix(test_label, test_pred, labels=range(9))
    
        print(train_result_cm, '\n', test_result_cm)


In [27]:
# 参数
PARAMS = {
        'pkl_name': 'data/label8_eurusd_10bar_1500_500_val200_gaf_culr.pkl',
        'model_name': 'model/model.h5',
        'classes': 9,
        'learning_rate': 0.01,
        'epochs': 50,
        'batch_size': 64,
        'optimizer': optimizers.legacy.SGD(learning_rate=0.01)
    }

# 加载数据
data = pro.load_pkl(PARAMS['pkl_name'])

In [28]:
# Initialize CNN Model Trainer
cnn_trainer = CNNModelTrainer(PARAMS)

# Train CNN model
model, hist = cnn_trainer.train_model(data)
    
# Save the trained model
model.save(PARAMS['model_name'])
  
# Print training and testing results
cnn_trainer.print_result(data)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 10, 10, 16)        272       
                                                                 
 activation_6 (Activation)   (None, 10, 10, 16)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 10, 10, 16)        1040      
                                                                 
 activation_7 (Activation)   (None, 10, 10, 16)        0         
                                                                 
 flatten_2 (Flatten)         (None, 1600)              0         
                                                                 
 dense_4 (Dense)             (None, 128)               204928    
                                                                 
 dense_5 (Dense)             (None, 9)                

/Users/syesw/anaconda3/envs/RL/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


157/157 [==============================] - 0s 574us/step
[[1980   97  157   98  171  187  136  107   67]
 [  82 1407    0    8    0    0    3    0    0]
 [  41    0 1435    0   24    0    0    0    0]
 [  55   39    0 1238    0    3    0  165    0]
 [  22    0   25    0 1397    0    3    0   53]
 [  48    3    0    1    0 1427    2   19    0]
 [  95    2    5    0    3    0 1345    0   50]
 [  31    7    0  146    0   26    0 1290    0]
 [  41    0    4    0  295    0   49    0 1111]] 
 [[666  32  60  29  59  65  25  41  23]
 [ 20 475   0   5   0   0   0   0   0]
 [  2   0 496   0   2   0   0   0   0]
 [ 18   9   0 423   0   0   0  50   0]
 [ 10   0  13   0 463   0   2   0  12]
 [ 11   1   0   0   0 483   0   5   0]
 [ 34   0   2   0   0   0 457   0   7]
 [  8   1   0  26   0  13   0 452   0]
 [ 18   0   0   0  79   0  14   0 389]]
